In [ ]:
# import glob
# import os
import pandas as pd
import altair as alt
import calendar
import datetime
from altair import pipe, limit_rows, to_values
import utm
import requests
import geopandas as gpd
import json
# Handle large data sets without embedding them in the notebook
# alt.data_transformers.enable('data_server')
# # Include an image for each plot since Gradescope only supports displaying plots as images
# alt.renderers.enable('mimetype')
# Change the max_rows limit of altair because our dataset is 56,000 rows.
t = lambda data: pipe(data, limit_rows(max_rows=60000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

# Reading in Data

In [ ]:
# Read in the data
url = 'https://raw.githubusercontent.com/sgskung/crime320/main/combined.csv'
df = pd.read_csv(url)

url2 = 'https://raw.githubusercontent.com/sgskung/crime320/main/incomes_ab.csv'
df2 = pd.read_csv(url2)

url3 = "http://geodash.vpd.ca/opendata/crimedata_download/crimedata_csv_all_years.csv"
df3 = pd.read_csv(url3)

In [ ]:
def crime_type(x):
    if 'Theft' in x:
        return 'Theft'
    elif 'Vehicle Collision' in x:
        return 'Vehicle Collision'
    elif 'Break and Enter' in x:
        return 'Break & Enter'
    else:
        return x

df['crime_type'] = df.apply(lambda x: crime_type(x['TYPE']), axis = 1)
df['crime_type'].unique()

counts = df.groupby(['crime_type']).count()
counts['type'] = counts.index
counts['count']= counts['TYPE']
counts =counts.drop(['TYPE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'HUNDRED_BLOCK', 'NEIGHBOURHOOD', 'X', 'Y'], axis=1)
counts['percent'] = round(((counts['count'] / 
                  counts['count'].sum())), 3)

selection = alt.selection_single()
# df
donut = alt.Chart(counts).mark_arc(innerRadius=100).add_selection(
        selection
    ).encode(
    theta=alt.Theta(field="count", type="quantitative"),
    color=alt.condition(selection, 'crime_type:N', alt.value('grey'), field="type"),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1)),
    tooltip=[
        alt.Tooltip('type:N'),
        alt.Tooltip('count:Q'),
        alt.Tooltip('percent:Q', format='.1%')
    ]
).properties(
    title='Breakdown of Crimes by Type'
)

In [ ]:
# Creating necessary variables
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# Convert column to non-scientific notation with two decimal places
df['Y'] = df['Y'].round(3).astype(float)

In [ ]:
type(utm.to_latlon(490079.3981, 5460209.675, 10, 'U'))

# df
# df["lat"] = df.apply(lambda x: utm.to_latlon(x['X'], x['Y'], 10, 'U')[0], axis = 1)

filtered_df = df[df['X'] > 0]
filtered_df.describe()
filtered_df["lat"] = filtered_df.apply(lambda x: utm.to_latlon(x['X'], x['Y'], 10, 'U')[0], axis = 1)
filtered_df["long"] = filtered_df.apply(lambda x: utm.to_latlon(x['X'], x['Y'], 10, 'U')[1], axis = 1)
filtered_df['name'] = filtered_df['NEIGHBOURHOOD']
# filtered_df
# for index, row in filtered_df.iterrows():
#     print("X")
#     print(row['X'])
#     print("Y")
#     print(row['Y'])
#     utm.to_latlon(row['X'], row['Y'], 10, 'U')

In [ ]:
def count_groups(dataframe, group_column):
    """
    Groups the rows in the given dataframe by the unique values in the specified group_column.
    Calculates the count of each group and creates a new dataframe with the name of the group and the count in each column.
    
    Args:
        dataframe (pandas.DataFrame): The input dataframe to group and count.
        group_column (str): The name of the column to group by.
    
    Returns:
        pandas.DataFrame: A new dataframe with two columns: the unique values in the group_column, and the count of each group.
    """
    
    # Group the dataframe by the unique values in the specified group column
    group_counts = dataframe.groupby(group_column).size().reset_index(name='count')
    
    # Return the new dataframe with the name of the group and the count in each column
    return group_counts

In [ ]:
df3.head(5)

In [ ]:
def get_most_common_type(df):
    # Group the dataframe by NEIGHBOURHOOD and TYPE and count the occurrences of each group
    grouped = df.groupby(['NEIGHBOURHOOD', 'TYPE']).size().reset_index(name='count')
    
    # Find the index of the row with the maximum count for each NEIGHBOURHOOD group
    idx = grouped.groupby('NEIGHBOURHOOD')['count'].transform(max) == grouped['count']
    
    # Subset the dataframe to the rows with the maximum count for each NEIGHBOURHOOD group
    result = grouped[idx][['NEIGHBOURHOOD', 'TYPE']]
    
    # Convert TYPE to string type
    result['TYPE'] = result['TYPE'].astype(str)
    
    # Group the result dataframe by NEIGHBOURHOOD and combine the TYPE values into a comma-separated string
    result = result.groupby('NEIGHBOURHOOD')['TYPE'].apply(lambda x: ', '.join(x)).reset_index()
    
    return result

y = get_most_common_type(df3)

In [ ]:
x = count_groups(df3, "NEIGHBOURHOOD")
x = x.merge(y, left_on='NEIGHBOURHOOD', right_on='NEIGHBOURHOOD', how='inner')
x = x.replace('Central Business District','Downtown')
x['MostCommon'] = x['TYPE']
x = x.drop(columns='TYPE')

# Choropleth Map + Donut

In [ ]:
url_geojson = "https://raw.githubusercontent.com/blackmad/neighborhoods/master/vancouver.geojson"
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features', type='json'))
van_hoods = "https://opendata.vancouver.ca/api/records/1.0/search/?dataset=local-area-boundary&q=&facet=name"
van_hoodss = 'local-area-boundary.shp'

In [ ]:
gdf = gpd.read_file(van_hoodss)

In [ ]:
gdf = gdf.merge(x, left_on='name', right_on='NEIGHBOURHOOD', how='inner')

In [ ]:
choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values=choro_json['features'])

In [ ]:
def gen_map(geodata, color_column, title, tooltip, color_scheme='bluegreen'):
    
    selection = alt.selection_multi(fields=[color_column])
    color = alt.condition(selection,
              alt.Color(color_column, type='nominal',
              scale=alt.Scale(scheme=color_scheme)),
              alt.value('lightgray'))
    
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    ).project('mercator')
    
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        stroke='black'
    ).encode(
         color = color,
         tooltip = tooltip
    ).project('mercator'
    ).add_selection(
        selection
    )
    
    donut2 = alt.Chart(geodata).mark_arc(innerRadius=100
    ).encode(
    theta = alt.Theta(field="properties.count", type="quantitative", sort=alt.EncodingSortField(order='ascending')),
    color = color,
    tooltip=[
        alt.Tooltip(color_column, type='nominal'),
        alt.Tooltip('properties.count:Q')
    ]
    ).properties(
        title='Breakdown of Crimes by Neighbourhood',
        height=350,
        width=300
    ).add_selection(
        selection
    )
    
    return alt.layer(base, choro) | donut2

In [ ]:
# crimes = alt.topo_feature(url_geojson, feature = 'Neighbourhood')
# counts = count_groups(filtered_df, 'NEIGHBOURHOOD')

# map = alt.Chart(crimes).mark_geoshape(stroke='black', strokeWidth=1).encode(
#     color='count:Q'
# ).transform_lookup(
#     lookup='NEIGHBOURHOOD',
#     from_ = alt.LookupData(counts, key = 'placeholder', fields = ['NEIGHBOURHOOD', 'count'])
# ).properties(
#     width=500,
#     height=300
# ).project('mercator')
# map

In [ ]:
# gdf = gpd.GeoDataFrame.from_features((anc_json))

In [ ]:
# gdf = gdf.merge(filtered_df, on='name', how='inner')
# gdf = gdf.drop(columns = ['created_at', 'updated_at'])
# gdf['DATE'] = gdf['DATE'].astype(str)

In [ ]:
# choro_json = json.loads(gdf.to_json())
# choro_data = alt.Data(values=choro_json['features'])

In [ ]:
# Code adapted from Vega-Altair 5 Documentation: Geographic Plots

# crimes = alt.topo_feature(url_geojson, feature = 'Neighbourhood')
# click = alt.selection_multi(fields=['name'])


# alt.Chart(crimes).mark_geoshape(fill='lightgray', stroke='white').encode(
#     longitude='X:Q',
#     latitude='Y:Q'
# ).project()

# overlay = alt.Chart(hoods).mark_geoshape(stroke='black', fillOpacity=0).encode(
# ).project(
# ).properties(
#     width=400,
#     height=200
# )

# points = alt.Chart(filtered_df).mark_circle(opacity = 0.35).encode(
#     longitude='long:Q',
#     latitude='lat:Q',
#     size=alt.value(5)
# )

# .transform_lookup(
#     lookup='name',
#     from_=alt.LookupData(filtered_df, 'id', ['count()', 'NEIGHBOURHOOD'])

# .add_selection(click)

In [ ]:
# variable = 'DAY'

# line = alt.Chart(df).mark_line(
# ).encode(
#     alt.X(variable, type = 'ordinal'),
#     alt.Y('count():Q')
# ).properties(
#     width = 400,
#     height = 200
# )

In [ ]:
# alt.layer(bg, points)

In [ ]:
# timespan = ['Timespan']
# select_box = alt.binding_select(options=times, name='column')
# sel = alt.selection_single(fields=['column'], bind=select_box, init={'column': 'MONTH'})

# alt.Chart(df).mark_line().transform_fold(
#     times,
#     as_=['column', 'value']
# ).transform_filter(
#     sel
# ).encode(
#     alt.X('MONTH:O', type = 'ordinal'),
#     alt.Y('count():Q')
# ).add_selection(
#     sel
# )

# sel

# Line Chart

In [ ]:
# Code adapted from @joelostblom StackOverflow answer: "Make dropdown selection responsive for y axis Altair python"

times = ['MONTH', 'DAY', 'HOUR']
select_box = alt.binding_select(options=times, name='Timespan: ')
xcol_param = alt.param(value = 'MONTH', bind = select_box)

line = alt.Chart(df, title="Crime Occurrences by Time of Day").mark_line(color='red'
).encode(
    x=alt.X('x:Q', title=''),
    y=alt.Y('count():Q').title('Count of Crime')
).transform_calculate(
    x=f'datum[{xcol_param.name}]'
).add_params(
    xcol_param
)

# Bar Chart + Bubble

In [ ]:
neighbours = df['NEIGHBOURHOOD'].unique() # get unique field values
neighbours = list(filter(lambda d: d is not None, neighbours)) # filter out None values
neighbours.sort() # sort alphabetically

In [ ]:
# selectNeighbour = alt.selection_single(
#     name='Select', # name the selection 'Select'
#     fields=['NEIGHBOURHOOD'], # limit selection to the Major_Genre field
#     init={'NEIGHBOURHOOD': neighbours[0]}, # use first genre entry as initial value
#     bind=alt.binding_select(options=neighbours) # bind to a menu of unique genre values
# )

# alt.Chart(df).mark_bar().add_selection(
#     selectNeighbour
# ).encode(
#     x='NEIGHBOURHOOD:N',
#     y='count():Q',
#     opacity=alt.condition(selectNeighbour, alt.value(1), alt.value(0.05))
# )

In [ ]:
crime_type_count_bar = alt.Chart(df).mark_bar().encode(    
    alt.X("NEIGHBOURHOOD", sort = '-y'),
    alt.Y("count()"),
    alt.Color('NEIGHBOURHOOD', scale=alt.Scale(scheme='purpleorange')),
    #alt.Tooltip('TYPE')
).properties(
    height = 275,
    width = 300
)

In [ ]:
#join median income column to df

# extracted_income = df2['Median Household Income After Taxes']
# df_w_income = df.join(extracted_income)
d = {}
for index, row in df2.iterrows():
    d[row['Neighborhood']] = row['Median Household Income After Taxes']
    # print(row['Neighborhood'])

df['Income'] = df.apply(lambda x: d[x['NEIGHBOURHOOD']], axis = 1)

In [ ]:
crime_bubble = alt.Chart(df).mark_circle().encode(    
    alt.X("Income", axis = None),
    alt.Y("count()", axis = None),
    alt.Color('TYPE', scale=alt.Scale(scheme='purpleorange')),
    alt.Tooltip('Income'),
    alt.Size("count()", scale=alt.Scale(range=[10, 500]))
).properties(
    height = 300,
    width = 350
)

In [ ]:
# Bubble
# .configure_axis(
#     grid = False
# ).configure_view(
#     strokeWidth = 0


# Bar
# .configure_axis(
#     grid = False
# )

In [ ]:
# Code adapted from Vega-Altair 5: Interaction Documentation
 
selection = alt.selection_point(fields=['NEIGHBOURHOOD'])
color = alt.condition(selection,
                      alt.Color('NEIGHBOURHOOD:N', scale=alt.Scale(scheme='purpleorange')),
                      alt.value('lightgray'))

crime_type = alt.Chart(df).mark_bar().encode(    
    alt.X("NEIGHBOURHOOD", sort = '-y'),
    alt.Y("count()"),
    color = color,
    #alt.Tooltip('TYPE')
).properties(
    height = 275,
    width = 300
).add_params(
    selection
)

crime_bub = alt.Chart(df).mark_circle().encode(    
    alt.X("Income", axis = None),
    alt.Y("count()", axis = None),
    alt.Tooltip('Income'),
    alt.Size("count()", scale=alt.Scale(range=[10, 500])),
    color = color
).properties(
    height = 300,
    width = 350
)

# All Visualizations

In [ ]:
van_map = gen_map(geodata=choro_data,
                      color_column='properties.name',
                      title=f'Crime by Location',
                      tooltip=['properties.name:O', 'properties.MostCommon:N'])
van_map

In [ ]:
crime_bub | crime_type

In [ ]:
line